In [1]:
%matplotlib inline
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
from scipy.stats import uniform as sp_randFloat
from sklearn import svm
from tqdm import tqdm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from time import time
from tabulate import tabulate
import numpy as np
import pandas as pd
import sklearn
import warnings
warnings.filterwarnings('ignore')
import os

In [2]:
from scipy.stats import randint as sp_randInt

from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import make_scorer
from scipy import sparse

In [3]:
import json
with open('GA_output_ET.json', 'r') as fp:
    feature_list = json.load(fp)

In [4]:
feature_list

{'SYN': ['IP_flags',
  'IP_DF',
  'TCP_dataofs',
  'TCP_SYN',
  'TCP_ACK',
  'sport_class',
  'dst_port_diversity',
  'src_IP_diversity',
  'IP_add_count',
  'pck_size_mean_WE',
  'pck_size_sum_of_EW',
  'ts_mean_WE',
  'ts_std_WE',
  'ts_sum_of_EW',
  'TCP_window_std_WE',
  'pck_size_mean_2',
  'TCP_ACK_sum',
  'TCP_ACK_ratio',
  'TCP_SYN_SR',
  'TCP_ACK_SR',
  'pck_size_mean_6',
  'ts_std_6',
  'ts_mean_9',
  'ts_std_9',
  'Label'],
 'HTTP': ['ts',
  'TCP_flags',
  'dport_class',
  'pck_size_sum_of_EW',
  'ts_sum_of_EW',
  'entropy_mean_WE',
  'TCP_window_mean_2',
  'sport_sum',
  'TCP_ACK_sum',
  'sum',
  'TCP_PSH_SR',
  'TCP_ACK_SR',
  'ts_std_6',
  'TCP_window_mean_6',
  'Label'],
 'ACK': ['TCP_window_sum_of_EW',
  'payload_bytes_mean_WE',
  'sport_sum',
  'TCP_ACK_sum',
  'TCP_ACK_ratio',
  'ts_mean_6',
  'Label'],
 'UDP': ['IP_flags',
  'IP_DF',
  'TCP_dataofs',
  'TCP_flags',
  'TCP_ACK',
  'TCP_window',
  'TCP_options',
  'sport_class',
  'Protocol',
  'TCP_window_std_WE',
  '

In [5]:
file_list={"SYN":['./INPUT/SM/DoS-SYN-1.csv','./INPUT/SM/DoS-SYN-2.csv'],
"HTTP":['./INPUT/SM/MB-HTTP-4.csv','./INPUT/SM/MB-HTTP-1.csv'],
"ACK":['./INPUT/SM/MB-ACK-4.csv','./INPUT/SM/MB-ACK-1.csv'],
"UDP":['./INPUT/SM/MB-UDP-4.csv','./INPUT/SM/MB-UDP-1.csv'],
"ARP":['./INPUT/SM/MitM-ARP-6.csv','./INPUT/SM/MitM-ARP-4.csv'],
"SP":['./INPUT/SM/Scan-Port-3.csv','./INPUT/SM/Scan-Port-4.csv'],
"BF":['./INPUT/SM/MB-BF-5.csv','./INPUT/SM/MB-BF-3.csv'],
"OS":['./INPUT/SM/Scan-OS-3.csv','./INPUT/SM/Scan-OS-6.csv'],
"SCHD":['./INPUT/SM/Scan-HDis-3.csv','./INPUT/SM/Scan-HDis-4.csv'],
"MHDis":['./INPUT/SM/MB-HDis-3.csv','./INPUT/SM/MB-HDis-2.csv']}

In [6]:
def run_random_search(model, params, x_train, y_train):
    #grid = GridSearchCV(model, params, cv = ps, n_jobs = -1, scoring = score, verbose = 0, refit = False)
    grid =RandomizedSearchCV(model, param_grid, cv=ps,scoring = 'f1_macro')
    grid.fit(x_train, y_train)
    return (grid.best_params_, round(grid.best_score_,8),grid.best_estimator_)

In [7]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

# RandomizedSearchCV  DT

In [8]:
lines=[['criterion', 'max_depth', 'max_features', 'min_samples_split', "F1","Std","Time","No","Attack"]]

for j in file_list:
    print(j)
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    
    print ('%-35s %-20s %-8s %-8s' % ("HYPERPARAMETERS","F1 Score", "Time", "No"))

    nfolds=10
    param_grid = { 'criterion':['gini','entropy'],
                      "max_depth":np.linspace(1, 32, 32, endpoint=True).astype(int),
                     "min_samples_split": sp_randint(2,10),#uniform(0.1,1 ),
                        # "min_samples_leafs" : np.linspace(0.1, 0.5, 5, endpoint=True)
                        "max_features" : sp_randint(1,X_train.shape[1])}

    second=time()
    f1=[]
    clf=DecisionTreeClassifier()
    for ii in range(25):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    for i in tqdm(range(50)):
        second=time()
        a,b,clf=run_random_search(DecisionTreeClassifier(),param_grid,X,y)
        f1=[]
        for ii in range(10):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:
results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("DT_HPO.csv",index=False)

final_parametres=[['criterion', 'max_depth', 'max_features', 'min_samples_split', "F1","Std","Time","No","Attack"]]
for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    m=df["max_depth"].min()
    df=df[df["max_depth"]==m]  
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))
    
    

SYN
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.9998489480892206   0.487    24      


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:33<00:00,  1.49it/s]


HTTP
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  1.967    24      


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [01:04<00:00,  1.28s/it]


ACK
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  1.115    24      


...

SCHD
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  1.058    24      


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:32<00:00,  1.55it/s]


MHDis
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.698    24      


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:23<00:00,  2.09it/s]

    criterion      max_depth    max_features    min_samples_split        F1    Std    Time    No  Attack
--  -----------  -----------  --------------  -------------------  --------  -----  ------  ----  --------
 0  entropy                9              25                    2  0.999849      0   0.926    38  SYN
 1  gini                   5              11                    4  1             0   1.38     28  HTTP
 2  entropy                4               3                    4  1             0   0.617    26  ACK
 3  entropy                5              18                    4  1             0   1.44     37  UDP
 4  entropy               12              60                    2  1             0   9.692    12  ARP
 5  entropy                5              10                    3  1             0   0.445    36  SP
 6  entropy                4              19                    4  1             0   0.951    17  BF
 7  entropy                2              28                    3  1       

# RandomizedSearchCV RF

In [9]:
lines=[['bootst', 'criter', 'max_depth', 'max_features',"min_samp_split","n_estimators", "F1","Std","Time","No","Attack"]]



for j in file_list:
    print(j)
    
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    
    
    
    
    print ('%-35s %-20s %-8s %-8s' % ("HYPERPARAMETERS","F1 Score", "Time", "No"))



    # use a full grid over all parameters
    param_grid = {"max_depth":np.linspace(1, 32, 32, endpoint=True).astype(int),
                  "n_estimators" : sp_randint(1, 200),
                  "max_features": sp_randint(1, 11),
                  "min_samples_split":sp_randint(2, 11),
                  "bootstrap": [True, False],
                  "criterion": ["gini", "entropy"]}
    second=time()
    f1=[]
    clf=RandomForestClassifier()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))


    
    ######################################################################################################################
    for i in tqdm(range(10)):
        second=time()
        a,b,clf=run_random_search(RandomForestClassifier(),param_grid,X,y)
        f1=[]
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:
results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("RF_HPO.csv",index=False)

final_parametres=[['bootst', 'criter', 'max_depth', 'max_features',"min_samp_split","n_estimators", "F1","Std","Time","No","Attack"]]

for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    m=df["max_depth"].min()
    df=df[df["max_depth"]==m]  
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    

SYN
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.9998489480892209   0.999    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [03:09<00:00, 18.98s/it]


HTTP
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  1.421    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [05:17<00:00, 31.72s/it]


ACK
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.892    0       


...

SCHD
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.644    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:09<00:00, 12.90s/it]


MHDis
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.453    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:15<00:00, 13.53s/it]

    bootst    criter      max_depth    max_features    min_samp_split    n_estimators        F1    Std    Time    No  Attack
--  --------  --------  -----------  --------------  ----------------  --------------  --------  -----  ------  ----  --------
 0  False     entropy             7               8                 3              35  0.999849      0  17.214     7  SYN
 1  False     gini               10               5                 6             114  1             0  40.993     1  HTTP
 2  True      entropy             7               8                 5             155  1             0  40.807     2  ACK
 3  False     entropy             4              10                 8             192  1             0  51.821     2  UDP
 4  True      entropy            20               3                 2             132  1             0  76.439     9  ARP
 5  False     entropy             7               3                 4              66  1             0  13.872     2  SP
 6  True      en

# RandomizedSearchCV  KNeighborsClassifier

In [11]:
lines=[['algorithm', 'leaf_size', 'n_neighbors', 'weights', "F1","Std","Time","No","Attack"]]


for j in file_list:
    print(j)
    
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    

    # use a full grid over all parameters
    param_grid = {"n_neighbors" : sp_randint(1,64) ,  
                 "leaf_size": sp_randint(1,50) , 
                  "algorithm" : ["auto", "ball_tree", "kd_tree", "brute"],
                  "weights" : ["uniform", "distance"]}
    second=time()
    f1=[]
    clf=KNeighborsClassifier()
    for ii in range(25):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))


    
    ######################################################################################################################
    for i in tqdm(range(10)):
        second=time()
        a,b,clf=run_random_search(KNeighborsClassifier(),param_grid,X,y)
        f1=[]
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:
results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("knn_HPO.csv",index=False)

final_parametres=[['algorithm', 'leaf_size', 'n_neighbors', 'weights', "F1","Std","Time","No","Attack"]]

for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    m=df["n_neighbors"].min()
    df=df[df["n_neighbors"]==m]  
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))
    
    
    
    
    

SYN
default                             0.9991944774663929   94.734   24      


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [11:05<00:00, 66.59s/it]


HTTP
default                             0.9934973725158257   19.98    24      


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [03:25<00:00, 20.53s/it]


ACK
default                             0.9063155626124317   16.543   24      


...

SCHD
default                             0.9978591636694685   29.902   24      


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [03:02<00:00, 18.22s/it]


MHDis
default                             0.96912274523627     29.221   24      


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:49<00:00, 10.99s/it]

    algorithm      leaf_size    n_neighbors  weights          F1    Std    Time    No  Attack
--  -----------  -----------  -------------  ---------  --------  -----  ------  ----  --------
 0  brute                 43              3  distance   0.999849      0  57.434     2  SYN
 1  auto                  31              3  distance   1             0  23.404     7  HTTP
 2  ball_tree              2              3  distance   1             0  28.837     5  ACK
 3  ball_tree              1              2  uniform    1             0  52.51      3  UDP
 4  kd_tree               22              5  distance   1             0  67.274     0  ARP
 5  auto                   2              1  uniform    1             0  12.807     8  SP
 6  kd_tree                2              1  distance   1             0  11.143     4  BF
 7  kd_tree               38              1  distance   1             0  10.644     2  OS
 8  kd_tree               11              2  uniform    1             0  11.404     

In [12]:
from sklearn.naive_bayes import GaussianNB

In [13]:
lines=[['var_smoothing', "F1","Std","Time","No","Attack"]]
for j in file_list:
    print(j)
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    second=time()

    param_grid = {
        'var_smoothing': np.logspace(0,-9, num=100),
    }

 
    second=time()
    f1=[]
    clf=GaussianNB()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    for i in tqdm(range(10)):
        second=time()
        a,b,clf=run_random_search(GaussianNB(),param_grid,X,y)
        f1=[]
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)


results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("NB_HPO.csv",index=False)
print (tabulate(results, headers=list(results.columns)))
    


SYN
default                             0.9903309606417221   0.05     0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.48it/s]


HTTP
default                             0.632531783845639    0.02     0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.34it/s]


ACK
default                             0.680330164197769    0.015    0       


...

SCHD
default                             0.9304361169451092   0.029    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  2.98it/s]


MHDis
default                             0.7116504442546024   0.019    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  3.61it/s]

      var_smoothing        F1          Std    Time    No  Attack
--  ---------------  --------  -----------  ------  ----  --------
 0      2.31013e-09  0.990131  0             0.961     0  SYN
 1      1.51991e-09  0.990181  1.11022e-16   0.935     1  SYN
 2      6.57933e-09  0.988825  1.11022e-16   0.715     2  SYN
 3      2.84804e-09  0.98988   0             0.616     3  SYN
 4      1.23285e-09  0.990332  0             0.596     4  SYN
 5      1e-08        0.988172  0             0.587     5  SYN
 6      2.84804e-09  0.98988   0             0.604     6  SYN
 7      2.31013e-07  0.975586  0             0.58      7  SYN
 8      1.51991e-08  0.98752   0             0.561     8  SYN
 9      4.32876e-09  0.989328  1.11022e-16   0.59      9  SYN
10      1.23285e-09  0.632425  0             0.446     0  HTTP
11      5.3367e-06   0.568824  0             0.437     1  HTTP
12      5.3367e-09   0.630514  0             0.43      2  HTTP
13      1.51991e-09  0.631945  0             0.4       3  H

In [14]:

final_parametres=[["var_smoothing","F1","Std","Time","No","Attack"]]
for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))
    

      var_smoothing        F1    Std    Time    No  Attack
--  ---------------  --------  -----  ------  ----  --------
 0      1.23285e-09  0.990332      0   0.596     4  SYN
 1      1e-09        0.632532      0   0.428     5  HTTP
 2      0.432876     0.898893      0   0.261     7  ACK
 3      1.23285e-09  0.67223       0   0.72      1  UDP
 4      1e-09        0.610349      0   1.2       7  ARP
 5      1e-09        0.982343      0   0.145     0  SP
 6      1e-07        0.775546      0   0.299     1  BF
 7      1.51991e-09  1             0   0.695     3  OS
 8      1e-09        0.930436      0   0.33      2  SCHD
 9      1e-09        0.71165       0   0.26      3  MHDis


# LR

In [15]:
from scipy.stats import loguniform
from sklearn.linear_model import LogisticRegression
lines=[["C",'penalty','Solver',  "F1","Std","Time","No","Attack"]]
for j in file_list:
    print(j)
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    second=time()

    param_grid = {
     'solver': ['newton-cg', 'lbfgs', 'liblinear'],
    'penalty' : ['none', 'l1', 'l2', 'elasticnet'],
    'C' : loguniform(1e-5, 100)
        }
    

 
    second=time()
    f1=[]
    clf=LogisticRegression()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    for i in tqdm(range(10)):
        second=time()
        a,b,clf=run_random_search(LogisticRegression(),param_grid,X,y)
        f1=[]
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)


results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("LR_HPO.csv",index=False)
print (tabulate(results, headers=list(results.columns)))
    


SYN
default                             0.9977348992015422   0.502    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:37<00:00,  3.77s/it]


HTTP
default                             0.9987025785149208   0.407    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:43<00:00, 10.39s/it]


ACK
default                             1.0                  0.545    0       


...

SCHD
default                             0.9962644097356761   0.25     0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:28<00:00,  2.89s/it]


MHDis
default                             1.0                  0.278    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [05:38<00:00, 33.90s/it]

               C  penalty    Solver           F1          Std     Time    No  Attack
--  ------------  ---------  ---------  --------  -----------  -------  ----  --------
 0   0.602325     l1         liblinear  0.999648  1.11022e-16    2.433     0  SYN
 1  73.4794       l2         lbfgs      0.998037  0              5.87      1  SYN
 2   0.00201112   l1         liblinear  0.995281  0.000955693    1.886     2  SYN
 3   0.331891     l1         liblinear  0.999648  2.36511e-09    3.349     3  SYN
 4   0.042147     l1         liblinear  0.99851   0.000359536    3.074     4  SYN
 5   0.0846503    l1         liblinear  0.999537  2.01368e-05    2.579     5  SYN
 6   0.591983     l2         lbfgs      0.997735  1.11022e-16    5.46      6  SYN
 7   0.659214     l2         lbfgs      0.998138  1.11022e-16    5.62      7  SYN
 8   7.8055       l2         lbfgs      0.997685  0              4.512     8  SYN
 9   0.0669816    l1         liblinear  0.999517  2.46615e-05    2.903     9  SYN
10  79.1

In [16]:
final_parametres=[["C",'penalty','Solver',  "F1","Std","Time","No","Attack"]]
for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))
    

               C  penalty    Solver           F1          Std     Time    No  Attack
--  ------------  ---------  ---------  --------  -----------  -------  ----  --------
 0   0.331891     l1         liblinear  0.999648  2.36511e-09    3.349     3  SYN
 1  79.1689       l2         lbfgs      1         0              3.725     0  HTTP
 2  12.8184       l1         liblinear  0.999616  0            134.999     0  ACK
 3   0.275089     none       lbfgs      1         0             36.152     0  UDP
 4  13.4981       l1         liblinear  0.828585  0.00224721   476.507     3  ARP
 5   1.94584      none       lbfgs      0.999095  1.11022e-16    2.725     0  SP
 6   2.87076      l1         liblinear  1         0              3.382     6  BF
 7   0.0564484    l2         liblinear  1         0              1.691     0  OS
 8   1.52913      none       lbfgs      1         0              2.73      0  SCHD
 9   0.000436482  none       lbfgs      1         0              3.12      0  MHDis


# MLP

In [8]:
import scipy.stats
from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV


param_dist = [
    {'hidden_layer_sizes': (scipy.stats.randint(low=2**5, high=2**11+1).rvs(), )}
    for _ in range(100)] + [
    {'hidden_layer_sizes': (
            scipy.stats.randint(low=2 ** 5, high=2 ** 11 + 1).rvs(),
            scipy.stats.randint(low=2 ** 5, high=2 ** 11 + 1).rvs(),
    )} for i in range(100)
]

layers=[]

for i in param_dist:
    layers.append(i['hidden_layer_sizes'])



param_grid =  {
    'hidden_layer_sizes': layers,
    'solver': ['sgd', 'adam', 'lbfgs'],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001,0.001,0.01,0.1, 0.05],
    'learning_rate': ['constant','adaptive']
}


In [9]:
# MLP

import scipy.stats
from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV


param_dist = [
    {'hidden_layer_sizes': (scipy.stats.randint(low=2**5, high=2**11+1).rvs(), )}
    for _ in range(100)] + [
    {'hidden_layer_sizes': (
            scipy.stats.randint(low=2 ** 5, high=2 ** 11 + 1).rvs(),
            scipy.stats.randint(low=2 ** 5, high=2 ** 11 + 1).rvs(),
    )} for i in range(100)
]

layers=[]

for i in param_dist:
    layers.append(i['hidden_layer_sizes'])



param_grid =  {
    'hidden_layer_sizes': layers,
    'solver': ['sgd', 'adam', 'lbfgs'],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001,0.001,0.01,0.1, 0.05],
    'learning_rate': ['constant','adaptive']
}




lines=[['hidden_layer_sizes', 'solver','activation','alpha','learning_rate', "F1","Std","Time","No","Attack"]]


for j in file_list:
    print(j)
    
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    

    param_grid =  {
        'hidden_layer_sizes': layers,
        'solver': ['sgd', 'adam', 'lbfgs'],
        'activation': ['relu', 'tanh'],
        'alpha': [0.0001,0.001,0.01,0.1, 0.05],
        'learning_rate': ['constant','adaptive']
    }



    second=time()
    f1=[]
    clf=MLPClassifier()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))


    
    ######################################################################################################################
    for i in tqdm(range(1)):
        second=time()
        a,b,clf=run_random_search(MLPClassifier(),param_grid,X,y)
        f1=[]
        for ii in range(1):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:

results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("ann_HPO.csv",index=False)


print (tabulate(results, headers=list(results.columns)))
    
    

SYN
default                             0.9994461615882726   9.114    0       


100%|█████████████████████████████████████████| 1/1 [1:19:38<00:00, 4778.30s/it]

{'solver': 'adam', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (1885, 1004), 'alpha': 0.01, 'activation': 'relu'} 0.9993958379274824   4778.3   0       
HTTP


default                             0.9983694657282158   5.787    0       


100%|███████████████████████████████████████████| 1/1 [35:47<00:00, 2147.56s/it]

...

100%|████████████████████████████████████████████| 1/1 [15:11<00:00, 911.76s/it]

{'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': (59,), 'alpha': 0.0001, 'activation': 'relu'} 0.9952063707856353   911.754  0       
MHDis


default                             0.9904541255754638   1.003    0       


100%|████████████████████████████████████████████| 1/1 [09:29<00:00, 569.91s/it]

{'solver': 'lbfgs', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (100, 1368), 'alpha': 0.1, 'activation': 'tanh'} 1.0                  569.912  0       
    hidden_layer_sizes    solver    activation      alpha  learning_rate          F1    Std      Time    No  Attack
--  --------------------  --------  ------------  -------  ---------------  --------  -----  --------  ----  --------
 0  adam                  adaptive  (1885, 1004)   0.01    relu             0.999396      0  4778.3       0  SYN
 1  adam                  constant  (1832, 332)    0.05    relu             0.994436      0  2147.56      0  HTTP
 2  lbfgs                 adaptive  (131, 377)     0.0001  tanh             1             0  2357.17      0  ACK
 3  adam                  adaptive  (930, 745)     0.001   tanh             1             0   400.477     0  UDP
 4  sgd                   constant  (911, 400)     0.0001  tanh             0.561542      0  1769.98      0  ARP
 5  sgd                   adaptive  (49, 

-------------

# RandomizedSearchCV  SVM

In [10]:
lines=[['C', 'gamma', "F1","Std","Time","No","Attack"]]


for j in file_list:
    print(j)
    
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    


    param_grid =  {'C': [0.001, 0.01, 0.1, 1, 10], 'gamma' : [0.001, 0.01, 0.1, 1]}  

    second=time()
    f1=[]
    clf=svm.SVC()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))


    
    ######################################################################################################################
    for i in tqdm(range(1)):
        second=time()
        a,b,clf=run_random_search(svm.SVC(),param_grid,X,y)
        f1=[]
        for ii in range(1):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:

results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("svm_HPO.csv",index=False)


print (tabulate(results, headers=list(results.columns)))
    
    

SYN
default                             0.9866339437470528   11.917   0       


100%|████████████████████████████████████████████| 1/1 [13:31<00:00, 811.90s/it]

{'gamma': 1, 'C': 1}                                                                       1.0                  811.898  0       
HTTP


default                             0.4897829026250669   3.039    0       


100%|████████████████████████████████████████████| 1/1 [12:15<00:00, 735.99s/it]

...

100%|████████████████████████████████████████████| 1/1 [01:45<00:00, 105.35s/it]

{'gamma': 1, 'C': 10}                                                                      0.9973265487625729   105.352  0       
MHDis


default                             0.49480455220188024  0.423    0       


100%|████████████████████████████████████████████| 1/1 [02:03<00:00, 123.63s/it]

{'gamma': 0.1, 'C': 10}                                                                    1.0                  123.626  0       
       C    gamma        F1    Std      Time    No  Attack
--  ----  -------  --------  -----  --------  ----  --------
 0  1           1  1             0   811.898     0  SYN
 1  1          10  1             0   735.99      0  HTTP
 2  1          10  1             0   385.279     0  ACK
 3  0.1        10  1             0   338.714     0  UDP
 4  0.1         1  1             0  1444.65      0  ARP
 5  1           1  1             0   195.879     0  SP
 6  0.01       10  1             0    62.145     0  BF
 7  0.1        10  1             0   263.464     0  OS
 8  1          10  0.997327      0   105.352     0  SCHD
 9  0.1        10  1             0   123.626     0  MHDis
